# Moving my save to a new account

UGH. When I decided to buy my own copy of Minecraft Java, I figured that as long as I was just playing single-player, it wouldn't be a big deal to switch my world over to my own account. And indeed, when I started the world as myself, I kept my levels, I kept my levels, I kept my inventory, I even kept everything in my ender chest.

But all my advancements were reset (meh), all my stats were zeroed out (feh), and, most upsettingly: _my cats had no idea who I was_ and refused to be right-clicked.

**That cannot stand**

I'd hoped that renaming some files and changing some values in `level.dat` would be sufficient to transfer everything over, and that got me my advancements back, but, as I'd feared, that didn't get my pets to love me again (I also assume that my villager reputation is back to zero, but since I haven't cured anyone yet, that's whatever).

So my goal for this notebook is to try out the [`python-NBT` library](https://github.com/twoolie/NBT), do some exploring, and find out how hard a Find-Replace will be for a world that, uncompressed, is about 1 GB in size.

## Imports, Setup and Macros

In [1]:
import json
from os import environ
from pathlib import Path
from typing import Any, Collection, Dict

import mutf8
import pandas as pd
from IPython.display import Markdown, display
from nbt import nbt, region

In [2]:
def format_file_size(path: Path) -> str:
    """Print the size of the specified file in
    human-readible form (KB / MB / GB)

    Parameters
    ----------
    path : Path
        The path to the file

    Returns
    -------
    str
        A prettily formatted file size

    Notes
    -----
    I would be shocked if there isn't a utility already built
    into the standard library to do this, but all I could find
    via Googling was a bunch of recipes and examples
    """
    size = path.stat().st_size  # in bytes
    for unit in ("B", "KB", "MB", "GB"):
        if size < 1024 / 2:
            return f"{size:.1f} {unit}"
        size = size / 1024
    return f"{size} TB"

In [3]:
def summarize_keystore(keystore: Dict[str, Any]) -> None:
    """Display a summary of the contents of a key-value store

    Parameters
    ----------
    keystore : dict
        The keystore to summarize

    Returns
    -------
    None
    """

    def _summarize_keystore(keystore: Dict[str, Any]) -> str:
        summary = ""
        for k, v in keystore.items():
            summary += f"\n - `{k}` : "
            if isinstance(v, (str, nbt.TAG_String)):
                summary += f'`"{v}"`'
            elif not isinstance(v, Collection):
                summary += f"`{str(v)}`"
            else:
                length = len(v)
                if 0 < length < 3:
                    summary += "\n"
                    if not isinstance(v, Dict):
                        v = {i: item for i, item in enumerate(v)}
                    summary += "\n".join(
                        (f"\t{line}" for line in _summarize_keystore(v).split("\n"))
                    )
                else:
                    summary += f"({len(v)} items)"
        return summary

    display(Markdown(_summarize_keystore(keystore)))

In [4]:
save_folder = Path(environ["SAVE_PATH"])

# make sure this is set correctly
for path in sorted(save_folder.glob("*")):
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")

- DIM-1 (folder)
- DIM1 (folder)
- advancements (folder)
- data (folder)
- datapacks (folder)
- entities (folder)
- icon.png (8.6 KB)
- level.dat (9.8 KB)
- level.dat_old (9.8 KB)
- playerdata (folder)
- poi (folder)
- region (folder)
- serverconfig (folder)
- session.lock (3.0 B)
- stats (folder)


## Loading `level.dat`

And--no worries--_backups have been made_.

In [5]:
%%time
level = nbt.NBTFile(save_folder / "level.dat")

CPU times: user 11.3 ms, sys: 0 ns, total: 11.3 ms
Wall time: 10.4 ms


In [6]:
print(f"There are {len(level['Data'])} keys in this NBT file")

There are 43 keys in this NBT file


Let's list all the keys, see what we're working with.

In [7]:
summarize_keystore(level["Data"])


 - `WanderingTraderSpawnChance` : `75`
 - `BorderCenterZ` : `0.0`
 - `Difficulty` : `3`
 - `BorderSizeLerpTime` : `0`
 - `raining` : `0`
 - `Time` : `9824271`
 - `GameType` : `0`
 - `ServerBrands` : 
	
	 - `0` : `"forge"`
	 - `1` : `"fabric"`
 - `BorderCenterX` : `0.0`
 - `BorderDamagePerBlock` : `0.2`
 - `BorderWarningBlocks` : `5.0`
 - `WorldGenSettings` : (4 items)
 - `DragonFight` : (4 items)
 - `BorderSizeLerpTarget` : `59999968.0`
 - `Version` : (4 items)
 - `DayTime` : `13714829`
 - `initialized` : `1`
 - `WasModded` : `1`
 - `allowCommands` : `0`
 - `WanderingTraderSpawnDelay` : `9600`
 - `CustomBossEvents` : (0 items)
 - `GameRules` : (35 items)
 - `Player` : (44 items)
 - `SpawnY` : `68`
 - `rainTime` : `44337`
 - `thunderTime` : `54095`
 - `SpawnZ` : `144`
 - `hardcore` : `0`
 - `WanderingTraderId` : (4 items)
 - `DifficultyLocked` : `0`
 - `SpawnX` : `-592`
 - `clearWeatherTime` : `0`
 - `thundering` : `0`
 - `SpawnAngle` : `0.0`
 - `version` : `19133`
 - `BorderSafeZone` : `5.0`
 - `LastPlayed` : `1644697632798`
 - `BorderWarningTime` : `15.0`
 - `ScheduledEvents` : (13 items)
 - `LevelName` : `"Esha Ness"`
 - `BorderSize` : `59999968.0`
 - `DataVersion` : `2865`
 - `DataPacks` : 
	
	 - `0` : `"Enabled"`
	 - `1` : `"Disabled"`

Kinda hilarious that the only entity that's called out in this top-level is the friggin' wandering trader.

Let's look into the "Player"

In [8]:
summarize_keystore(level["Data"]["Player"])


 - `Brain` : 
	
	 - `0` : `"memories"`
 - `HurtByTimestamp` : `25043`
 - `SleepTimer` : `0`
 - `SpawnForced` : `0`
 - `Attributes` : (6 items)
 - `Invulnerable` : `0`
 - `FallFlying` : `0`
 - `PortalCooldown` : `0`
 - `AbsorptionAmount` : `0.0`
 - `abilities` : (7 items)
 - `FallDistance` : `0.0`
 - `recipeBook` : (10 items)
 - `DeathTime` : `0`
 - `XpSeed` : `-1997504740`
 - `XpTotal` : `12545`
 - `UUID` : (4 items)
 - `playerGameType` : `0`
 - `SpawnDimension` : `"minecraft:overworld"`
 - `Tags` : 
	
	 - `0` : `"ch_south"`
 - `seenCredits` : `0`
 - `Motion` : (3 items)
 - `SpawnY` : `107`
 - `Health` : `20.0`
 - `SpawnZ` : `-3925`
 - `foodSaturationLevel` : `0.40000057220458984`
 - `SpawnX` : `-1506`
 - `Air` : `300`
 - `OnGround` : `1`
 - `Dimension` : `"minecraft:overworld"`
 - `SpawnAngle` : `-81.14228057861328`
 - `Rotation` : 
	
	 - `0` : `26.203399658203125`
	 - `1` : `20.099943161010742`
 - `XpLevel` : `32`
 - `Score` : `12545`
 - `Pos` : (3 items)
 - `Fire` : `-20`
 - `XpP` : `0.05235851928591728`
 - `EnderItems` : (25 items)
 - `DataVersion` : `2865`
 - `foodLevel` : `20`
 - `foodExhaustionLevel` : `2.616138219833374`
 - `HurtTime` : `0`
 - `SelectedItemSlot` : `5`
 - `Inventory` : (25 items)
 - `foodTickTimer` : `0`

So it looks like the only bit of identifying informastion that's in here is my UUID (no name or anything like that). Let's look at that UUID:

In [ ]:
print(level["Data"]["Player"]["UUID"])

Four intergers. Interesting...

In [ ]:
print([hex(item % 2**32) for item in level["Data"]["Player"]["UUID"]])

Yeah, I've seen that in filenames.

In [11]:
old_uuid = tuple(v for v in level["Data"]["Player"]["UUID"])

## Now What Does Entity Data Look Like?

In [12]:
all_entitiy_files = sorted((save_folder / "entities").glob("*"))
for path in all_entitiy_files[:10]:
    print(f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})")
print(f"... {len(all_entitiy_files) - 10} more")

- r.-1.-1.mca (232.0 KB)
- r.-1.-10.mca (320.0 KB)
- r.-1.-11.mca (0.6 MB)
- r.-1.-12.mca (340.0 KB)
- r.-1.-2.mca (456.0 KB)
- r.-1.-3.mca (468.0 KB)
- r.-1.-4.mca (0.7 MB)
- r.-1.-5.mca (448.0 KB)
- r.-1.-6.mca (272.0 KB)
- r.-1.-7.mca (260.0 KB)
... 88 more


From [the wiki](https://minecraft.fandom.com/wiki/Entity_format#Entity_Format):
                
> Entities are stored in the entities folder of respective dimension folders. It is stored like regional Minecraft Anvil files, which are named in the form r.x.z.mca. 

And it looks like x/z are referncing the chunk numbers. So let's see... I have a bunch of cats somewhere around (550, 2500) which would be, like, (34, -157). So let's see... I should have a look at...

In [13]:
for path in all_entitiy_files:
    _, x, z, _ = path.name.split(".")
    if 30 < int(x) < 40 and -165 < int(z) < -150:
        print(
            f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})"
        )

Um... I definitely don't see any filenames anywhere near those numbers. Rgh. After some further digging, it appears that each "region" file is linked to a _collection_ of chunks, which the NBT library lets me access via `.iter_chunks()`.

In [14]:
%%time
for path in all_entitiy_files:
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        x, z = chunk["Position"]
        if 30 < x < 40 and -165 < z < -150:
            print(
                f"- {path.name} ({'folder' if path.is_dir() else format_file_size(path)})"
            )
            break

- r.0.-5.mca (460.0 KB)
- r.0.-6.mca (352.0 KB)
- r.1.-5.mca (0.5 MB)
- r.1.-6.mca (0.6 MB)
CPU times: user 2.46 s, sys: 129 ms, total: 2.59 s
Wall time: 2.59 s


Let's start with the biggest.

In [15]:
region_data = region.RegionFile(save_folder / "entities" / "r.1.-6.mca")
for chunk in region_data.iter_chunks():
    print(f"- Chunk {chunk.loc} has {len(chunk['Entities'])} entities")

- Chunk Location(x=0, y=None, z=4) has 1 entities
- Chunk Location(x=1, y=None, z=29) has 1 entities
- Chunk Location(x=1, y=None, z=30) has 5 entities
- Chunk Location(x=1, y=None, z=31) has 1 entities
- Chunk Location(x=2, y=None, z=7) has 1 entities
- Chunk Location(x=2, y=None, z=29) has 2 entities
- Chunk Location(x=2, y=None, z=30) has 2 entities
- Chunk Location(x=3, y=None, z=29) has 1 entities
- Chunk Location(x=3, y=None, z=30) has 1 entities
- Chunk Location(x=3, y=None, z=31) has 2 entities
- Chunk Location(x=4, y=None, z=29) has 1 entities
- Chunk Location(x=4, y=None, z=31) has 3 entities
- Chunk Location(x=5, y=None, z=28) has 1 entities
- Chunk Location(x=5, y=None, z=29) has 4 entities
- Chunk Location(x=5, y=None, z=30) has 8 entities
- Chunk Location(x=5, y=None, z=31) has 6 entities
- Chunk Location(x=6, y=None, z=28) has 1 entities
- Chunk Location(x=6, y=None, z=29) has 2 entities
- Chunk Location(x=6, y=None, z=30) has 1 entities
- Chunk Location(x=6, y=None, z=3

Again, follow my nose, go with the largest.

In [16]:
chunk = region_data.get_chunk(25, 4)
for entity in chunk["Entities"]:
    print(entity["id"])

minecraft:skeleton
minecraft:skeleton
minecraft:skeleton
minecraft:skeleton
minecraft:zombie
minecraft:villager
minecraft:villager
minecraft:villager
minecraft:villager


Well these aren't my kitties.

In [17]:
print(chunk["Position"])

[57, -188]


I mean, no wonder--this is pretty far away from where I was aiming. But this is a good chance to confirm I'm parsing this position data correctly. brb

Yeah, there's a village at (928, -3008).

Now, I don't particularly mind if my reputation gets reset with the villagers, but this is a good chance for me to see how entities register the player.

In [18]:
summarize_keystore(chunk["Entities"][-1])


 - `Brain` : 
	
	 - `0` : `"memories"`
 - `HurtByTimestamp` : `0`
 - `Attributes` : 
	
	 - `0` : 
		
		 - `0` : `"Base"`
		 - `1` : `"Name"`
	 - `1` : (3 items)
 - `FoodLevel` : `0`
 - `Invulnerable` : `0`
 - `FallFlying` : `0`
 - `ForcedAge` : `0`
 - `Gossips` : 
	
	 - `0` : (3 items)
 - `PortalCooldown` : `0`
 - `AbsorptionAmount` : `0.0`
 - `LastRestock` : `8153240`
 - `FallDistance` : `0.0`
 - `DeathTime` : `0`
 - `Xp` : `4`
 - `LastGossipDecay` : `8153238`
 - `HandDropChances` : 
	
	 - `0` : `0.08500000089406967`
	 - `1` : `0.08500000089406967`
 - `PersistenceRequired` : `0`
 - `id` : `"minecraft:villager"`
 - `UUID` : (4 items)
 - `Age` : `0`
 - `Motion` : (3 items)
 - `Health` : `20.0`
 - `LeftHanded` : `0`
 - `Air` : `300`
 - `OnGround` : `1`
 - `Offers` : 
	
	 - `0` : `"Recipes"`
 - `Rotation` : 
	
	 - `0` : `121.75462341308594`
	 - `1` : `0.0`
 - `HandItems` : 
	
	 - `0` : (0 items)
	 - `1` : (0 items)
 - `RestocksToday` : `0`
 - `ArmorDropChances` : (4 items)
 - `Pos` : (3 items)
 - `Fire` : `-1`
 - `ArmorItems` : (4 items)
 - `CanPickUpLoot` : `1`
 - `VillagerData` : (3 items)
 - `HurtTime` : `0`
 - `Inventory` : 
	
	 - `0` : 
		
		 - `0` : `"id"`
		 - `1` : `"Count"`
	 - `1` : 
		
		 - `0` : `"id"`
		 - `1` : `"Count"`

In [19]:
summarize_keystore(chunk["Entities"][-1]["VillagerData"])


 - `profession` : `"minecraft:librarian"`
 - `level` : `1`
 - `type` : `"minecraft:plains"`

In [20]:
summarize_keystore(chunk["Entities"][-1]["Gossips"][0])


 - `Target` : (4 items)
 - `Type` : `"trading"`
 - `Value` : `4`

In [21]:
tuple(v for v in chunk["Entities"][-1]["Gossips"][0]["Target"]) == old_uuid

True

Yep, so they're gossiping about me. But is that how it determines reputation?

### Anyway, back to the kitties

In [22]:
%%time
for path in all_entitiy_files:
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        x, z = chunk["Position"]
        if 30 < x < 40 and -165 < z < -150:
            print(
                f"- Chunk {chunk.loc} in {path.name} has {len(chunk['Entities'])} entities"
            )

- Chunk Location(x=31, y=None, z=0) in r.0.-5.mca has 1 entities
- Chunk Location(x=31, y=None, z=1) in r.0.-5.mca has 4 entities
- Chunk Location(x=31, y=None, z=3) in r.0.-5.mca has 1 entities
- Chunk Location(x=31, y=None, z=4) in r.0.-5.mca has 1 entities
- Chunk Location(x=31, y=None, z=5) in r.0.-5.mca has 1 entities
- Chunk Location(x=31, y=None, z=30) in r.0.-6.mca has 1 entities
- Chunk Location(x=0, y=None, z=3) in r.1.-5.mca has 1 entities
- Chunk Location(x=0, y=None, z=5) in r.1.-5.mca has 2 entities
- Chunk Location(x=1, y=None, z=1) in r.1.-5.mca has 1 entities
- Chunk Location(x=1, y=None, z=2) in r.1.-5.mca has 1 entities
- Chunk Location(x=1, y=None, z=4) in r.1.-5.mca has 1 entities
- Chunk Location(x=1, y=None, z=5) in r.1.-5.mca has 2 entities
- Chunk Location(x=2, y=None, z=1) in r.1.-5.mca has 18 entities
- Chunk Location(x=2, y=None, z=2) in r.1.-5.mca has 3 entities
- Chunk Location(x=2, y=None, z=3) in r.1.-5.mca has 2 entities
- Chunk Location(x=2, y=None, z=

Okay, yeah, let's look at that one with 18 entities. 

In [23]:
chunk = region.RegionFile(save_folder / "entities" / "r.1.-5.mca").get_chunk(2, 1)
for entity in chunk["Entities"]:
    print(entity["id"])

minecraft:item_frame
minecraft:parrot
minecraft:cat
minecraft:item_frame
minecraft:item_frame
minecraft:cat
minecraft:cat
minecraft:cat
minecraft:cat
minecraft:cat
minecraft:cat
minecraft:cat
minecraft:item_frame
minecraft:cat
minecraft:cat
minecraft:item_frame
minecraft:item_frame
minecraft:item_frame


KITTEHS!!!

In [24]:
summarize_keystore(chunk["Entities"][2])


 - `Brain` : 
	
	 - `0` : `"memories"`
 - `HurtByTimestamp` : `0`
 - `Owner` : (4 items)
 - `CatType` : `2`
 - `Sitting` : `1`
 - `Attributes` : 
	
	 - `0` : 
		
		 - `0` : `"Base"`
		 - `1` : `"Name"`
	 - `1` : (3 items)
 - `Invulnerable` : `0`
 - `FallFlying` : `0`
 - `ForcedAge` : `0`
 - `PortalCooldown` : `0`
 - `AbsorptionAmount` : `0.0`
 - `FallDistance` : `0.0`
 - `InLove` : `0`
 - `DeathTime` : `0`
 - `HandDropChances` : 
	
	 - `0` : `0.08500000089406967`
	 - `1` : `0.08500000089406967`
 - `PersistenceRequired` : `1`
 - `id` : `"minecraft:cat"`
 - `UUID` : (4 items)
 - `Age` : `0`
 - `CollarColor` : `14`
 - `Motion` : (3 items)
 - `Health` : `10.0`
 - `LeftHanded` : `0`
 - `Air` : `300`
 - `OnGround` : `1`
 - `Rotation` : 
	
	 - `0` : `171.1956787109375`
	 - `1` : `0.0`
 - `HandItems` : 
	
	 - `0` : (0 items)
	 - `1` : (0 items)
 - `ArmorDropChances` : (4 items)
 - `CustomName` : `"{"text":"The Perilous Poozer"}"`
 - `Pos` : (3 items)
 - `Fire` : `-1`
 - `ArmorItems` : (4 items)
 - `CanPickUpLoot` : `0`
 - `HurtTime` : `0`

**OWNER!!!!**

I want to take a look at memories for a second.

### How many pets do I have?

In [25]:
def get_name(entity: Dict[str, Any]) -> str:
    """Return the name (or identifier) of an entity

    Parameters
    ----------
    entity: dict
        The entity of interest

    Returns
    -------
    str
        The name and ID (or just the ID) of the entity
    """
    identifier = entity["id"]

    if "CustomName" not in entity.keys():
        return str(identifier)
    name = json.loads(entity["CustomName"].value)["text"]
    return f"{name} ({identifier})"

In [26]:
%%time
my_pets = []
for path in all_entitiy_files:
    region_data = region.RegionFile(path)
    for chunk in region_data.iter_chunks():
        for entity in chunk["Entities"]:
            try:
                owner = entity["Owner"]
            except KeyError:
                continue

            identifier = get_name(entity)

            position = ", ".join((str(int(float(str(v)))) for v in entity["Pos"]))

            if tuple(v for v in owner) != old_uuid:
                continue

            my_pets.append(entity)

            print(f"- Found {identifier} at ({position})")

- Found minecraft:horse at (-1520, 111, -3970)
- Found minecraft:donkey at (-1528, 110, -3963)
- Found minecraft:donkey at (-1525, 111, -3943)
- Found Mare-claren F1 (minecraft:horse) at (-1509, 111, -3955)
- Found Ridgeline (minecraft:donkey) at (-1504, 110, -3955)
- Found minecraft:horse at (-1518, 111, -3949)
- Found Ferrar-neigh (minecraft:horse) at (-1510, 111, -3946)
- Found minecraft:donkey at (-1504, 111, -3950)
- Found Nethernugg (minecraft:cat) at (-1504, 108, -3928)
- Found minecraft:cat at (-1505, 107, -3923)
- Found minecraft:horse at (-1492, 109, -3965)
- Found Horsche 911 (minecraft:horse) at (-1495, 110, -3952)
- Found minecraft:donkey at (-1496, 111, -3949)
- Found Lamborghi-neigh (minecraft:horse) at (-1496, 111, -3942)
- Found minecraft:horse at (-1108, 64, -4374)
- Found Jabber (minecraft:parrot) at (550, 78, -2541)
- Found The Perilous Poozer (minecraft:cat) at (550, 78, -2541)
- Found Cavecat (minecraft:cat) at (549, 73, -2532)
- Found Sam Phao (minecraft:cat) at 

Hi everybody!!

## Digression: Which Horse is the Best?

In [27]:
horse_data = []
for pet in my_pets:

    if pet["id"].value not in ("minecraft:horse", "minecraft:mule", "minecraft:donkey"):
        continue

    attributes = {"name": get_name(pet)}
    for attr in pet["Attributes"]:
        attributes[attr["Name"].value.split(":")[-1].split(".")[-1]] = attr[
            "Base"
        ].value

    horse_data.append(attributes)
horse_dataframe = pd.DataFrame(horse_data).set_index("name")
horse_dataframe

,follow_range,max_health,jump_strength,armor,movement_speed
name,,,,,
minecraft:horse,16.0,23.000000,0.640759,0.0,0.173832
minecraft:donkey,16.0,27.000000,NaN,NaN,0.175000
minecraft:donkey,16.0,25.000000,NaN,NaN,0.175000
Mare-claren F1 (minecraft:horse),16.0,28.000000,0.602987,0.0,0.292318
Ridgeline (minecraft:donkey),16.0,26.000000,NaN,NaN,0.175000
minecraft:horse,16.0,19.000000,0.839726,0.0,0.308744
Ferrar-neigh (minecraft:horse),16.0,29.000000,0.746220,0.0,0.182948
minecraft:donkey,16.0,15.000000,NaN,NaN,0.175000
minecraft:horse,16.0,22.000000,0.696182,0.0,0.244324


In [28]:
horse_dataframe.sort_values("max_health", ascending=False)

,follow_range,max_health,jump_strength,armor,movement_speed
name,,,,,
Ferrar-neigh (minecraft:horse),16.0,29.000000,0.746220,0.0,0.182948
Mare-claren F1 (minecraft:horse),16.0,28.000000,0.602987,0.0,0.292318
Horsche 911 (minecraft:horse),NaN,27.666667,0.711469,0.0,0.236387
minecraft:donkey,16.0,27.000000,NaN,NaN,0.175000
Lamborghi-neigh (minecraft:horse),16.0,27.000000,0.566055,0.0,0.271633
Ridgeline (minecraft:donkey),16.0,26.000000,NaN,NaN,0.175000
minecraft:donkey,16.0,25.000000,NaN,NaN,0.175000
minecraft:donkey,16.0,25.000000,NaN,NaN,0.175000
minecraft:horse,16.0,23.000000,0.640759,0.0,0.173832


Okay, so Ferrar-neigh is the chonkiest of chonks. Noted.

In [29]:
horse_dataframe.sort_values("jump_strength", ascending=False)

,follow_range,max_health,jump_strength,armor,movement_speed
name,,,,,
minecraft:horse,16.0,21.000000,0.910533,0.0,0.232124
minecraft:horse,16.0,19.000000,0.839726,0.0,0.308744
Ferrar-neigh (minecraft:horse),16.0,29.000000,0.746220,0.0,0.182948
Horsche 911 (minecraft:horse),NaN,27.666667,0.711469,0.0,0.236387
minecraft:horse,16.0,22.000000,0.696182,0.0,0.244324
minecraft:horse,16.0,23.000000,0.640759,0.0,0.173832
Mare-claren F1 (minecraft:horse),16.0,28.000000,0.602987,0.0,0.292318
Lamborghi-neigh (minecraft:horse),16.0,27.000000,0.566055,0.0,0.271633
minecraft:donkey,16.0,27.000000,NaN,NaN,0.175000


That's funny, because it's really been feeling like Lamborghi-neigh is a good jumper.

In [30]:
horse_dataframe.sort_values("movement_speed", ascending=False)

,follow_range,max_health,jump_strength,armor,movement_speed
name,,,,,
minecraft:horse,16.0,19.000000,0.839726,0.0,0.308744
Mare-claren F1 (minecraft:horse),16.0,28.000000,0.602987,0.0,0.292318
Lamborghi-neigh (minecraft:horse),16.0,27.000000,0.566055,0.0,0.271633
minecraft:horse,16.0,22.000000,0.696182,0.0,0.244324
Horsche 911 (minecraft:horse),NaN,27.666667,0.711469,0.0,0.236387
minecraft:horse,16.0,21.000000,0.910533,0.0,0.232124
Ferrar-neigh (minecraft:horse),16.0,29.000000,0.746220,0.0,0.182948
minecraft:donkey,16.0,27.000000,NaN,NaN,0.175000
minecraft:donkey,16.0,25.000000,NaN,NaN,0.175000


Wow. And I thought Mare-claren was fast! Super surprised that there's a horse that's faster (who, btw, is also the second best jumper, even though their health is for poo)

And then among the donks, I actually suspect I know who that 27 / 0.175000 beast is, and they're owed a nametag.

## Do I want to leave it at that?

I mean, from here I can go into the NBT data--either using the library or no--and alter the UUID. But do I want to keep going and see where else the UUID crops up?

## Final Note

I was expecting parsing through the entire save file was going to be computationally exhausting, but the longest searches took like three seconds. Possible I just have a good SSD and that this NBT library is written in a pretty efficent way. For documentation's sake:

In [31]:
!neofetch

         eeeeeeeeeeeeeeeee
      eeeeeeeeeeeeeeeeeeeeeee
    eeeee  eeeeeeeeeeee   eeeee
  eeee   eeeee       eee     eeee
 eeee   eeee          eee     eeee
eee    eee            eee       eee
eee   eee            eee        eee
ee    eee           eeee       eeee
ee    eee         eeeee      eeeeee
ee    eee       eeeee      eeeee ee
eee   eeee   eeeeee      eeeee  eee
eee    eeeeeeeeee     eeeeee    eee
 eeeeeeeeeeeeeeeeeeeeeeee    eeeee
  eeeeeeee eeeeeeeeeeee      eeee
    eeeee                 eeeee
      eeeeeee         eeeeeee
         eeeeeeeeeeeeeeeee
antar@snow-maiden 
----------------- 
OS: elementary OS 6.1 Jólnir x86_64 
Kernel: 5.13.0-28-generic 
Uptime: 7 days, 23 hours, 20 mins 
Packages: 2127 (dpkg), 34 (flatpak) 
Shell: bash 5.0.17 
Resolution: 1920x1080, 1280x720 
DE: Pantheon 
WM: Mutter(Gala) 
Theme: io.elementary.stylesheet.blueberry [GTK3] 
Icons: elementary [GTK3] 
Terminal: jupyter-lab 
CPU: AMD Ryzen 5 3400G (8) @ 3.700GHz 
GPU: AMD ATI 05:00.0 Picasso 
Memor